In [1]:
import "github.com/umbralcalc/stochadex/pkg/simulator"

%%
params := simulator.NewParams(make(map[string][]float64))
params.Set("chicken", []float64{1.0, 2.0})

In [10]:
import (
	"bufio"
	"encoding/json"
	"fmt"
	"os"
	"strconv"
	"strings"

	"github.com/go-gota/gota/dataframe"
	dataseries "github.com/go-gota/gota/series"
	"github.com/umbralcalc/stochadex/pkg/simulator"
)

// QueryLogEntry is the output format from the logs scanner.
type QueryLogEntry struct {
	PartitionIterations int                    `json:"partition_iterations"`
	Entry               simulator.JsonLogEntry `json:"entry"`
}

// QueryLogEntries maps each collection of QueryLogEntry structs to a partition.
type QueryLogEntries struct {
	EntriesByPartition map[string][]QueryLogEntry
}

// DataFrameFromPartition constructs a dataframe from the state values of
// a given partition.
func (q *QueryLogEntries) DataFrameFromPartition(
	partitionName string,
) dataframe.DataFrame {
	series := make([]dataseries.Series, 0)
	entries, ok := q.EntriesByPartition[partitionName]
	if !ok {
		partitions := make([]string, 0)
		for name := range q.EntriesByPartition {
			partitions = append(partitions, name)
		}
		panic("partition name: " + partitionName +
			" not found, choices are: " + strings.Join(partitions, ", "))
	}
	for i := 0; i < len(entries[0].Entry.State); i++ {
		series = append(
			series,
			dataseries.New([]float64{}, dataseries.Float, strconv.Itoa(i)),
		)
	}
	for _, entry := range entries {
		for i, value := range entry.Entry.State {
			series[i].Append(value)
		}
	}
	return dataframe.New(series...)
}

// ReadLogEntries reads a file up to a given number of iterations into
// a collection which maps each entry to a partition.
func ReadLogEntries(
	filename string,
	numIterations int,
) (*QueryLogEntries, error) {
	file, err := os.Open(filename)
	if err != nil {
		return nil, err
	}
	defer file.Close()

	// Loop through for the specified number of iterations
	partitionIterations := make(map[string]int)
	entriesByPartition := make(map[string][]QueryLogEntry)
	scanner := bufio.NewScanner(file)
	for scanner.Scan() {
		var logEntry simulator.JsonLogEntry
		line := scanner.Bytes()

		err := json.Unmarshal(line, &logEntry)
		if err != nil {
			fmt.Println("Error decoding JSON:", err)
			continue
		}

		// keep a track of how many iterations each partition has been through
		if iters, ok := partitionIterations[logEntry.PartitionName]; ok {
			partitionIterations[logEntry.PartitionName] = iters + 1
		} else {
			partitionIterations[logEntry.PartitionName] = 0
		}

		// Append the log entries
		if entries, ok := entriesByPartition[logEntry.PartitionName]; ok {
			entriesByPartition[logEntry.PartitionName] = append(
				entries,
				QueryLogEntry{
					PartitionIterations: partitionIterations[logEntry.PartitionName],
					Entry:               logEntry,
				},
			)
		} else {
			entriesByPartition[logEntry.PartitionName] = []QueryLogEntry{{
				PartitionIterations: partitionIterations[logEntry.PartitionName],
				Entry:               logEntry,
			}}
		}
		if partitionIterations[logEntry.PartitionName] > numIterations {
			break
		}
	}

	if err := scanner.Err(); err != nil {
		fmt.Println("Error reading file:", err)
	}

	return &QueryLogEntries{EntriesByPartition: entriesByPartition}, nil
}


%%
entries, _ := ReadLogEntries("../log/test.log", 100)
df := entries.DataFrameFromPartition("first_wiener_process")
fmt.Println(df)

[100x5] DataFrame

    0         1         2         3         4
 0: 0.649720  1.099574  -0.670531 0.083138  1.427430
 1: 2.371560  1.463875  0.581674  1.665532  0.676721
 2: 0.716977  -0.440915 1.357698  1.394333  -1.522020
 3: -0.215313 -1.043261 1.253280  0.519112  -2.322541
 4: -0.286350 -2.598860 -0.096717 1.138454  -2.643547
 5: -0.034675 -3.212641 -0.165612 1.871783  -3.146190
 6: 0.651897  -4.831194 0.477687  0.538956  -4.744302
 7: -0.246745 -5.406466 1.387489  2.637214  -3.307131
 8: -0.322987 -5.936725 2.185527  0.762030  -3.262084
 9: -0.605791 -8.673940 1.721032  -0.587701 -3.371957
    ...       ...       ...       ...       ...
    <float>   <float>   <float>   <float>   <float>



In [1]:
import (
	"fmt"
	"log"
	"os"
	"os/exec"
	"strings"
	"time"

	"github.com/go-gota/gota/dataframe"
	"github.com/go-echarts/go-echarts/v2/charts"
	"github.com/go-echarts/go-echarts/v2/opts"
	"github.com/go-echarts/go-echarts/v2/types"
    gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

func plotSomething() {
	csvStr := `
Country,Date,Age,Amount,Id
"United States",2012-02-01,50,112.1,01234
"United States",2012-02-02,32,321.31,54320
"United Kingdom",2012-02-03,17,18.2,12345
"United States",2012-02-04,32,321.31,54320
"United Kingdom",2012-02-05,NA,18.2,12345
"United States",2012-02-06,32,321.31,54320
"United States",2012-02-07,32,321.31,54320
"Spain",2012-02-08,66,555.42,00241
`

	// Load CSV data into dataframe
	df := dataframe.ReadCSV(strings.NewReader(csvStr))

	// Extract Date and Amount columns
	dateCol := df.Col("Date").Records()
	amountCol := df.Col("Amount").Float()

	// Prepare data for scatter plot
	var scatterData []opts.ScatterData
	for i, dateStr := range dateCol {
		date, err := time.Parse("2006-01-02", dateStr)
		if err != nil {
			log.Fatalf("error parsing date: %v", err)
		}
		scatterData = append(scatterData, opts.ScatterData{
			Value: []interface{}{date.Format("2006-01-02"), amountCol[i]},
		})
	}

	// Create scatter plot
	scatter := charts.NewScatter()
	scatter.SetGlobalOptions(
		charts.WithTitleOpts(opts.Title{Title: "Date vs Amount Scatter Plot"}),
		charts.WithXAxisOpts(opts.XAxis{
			Type: "category",
			Name: "Date",
			AxisLabel: &opts.AxisLabel{
				Rotate: 45, // Rotate labels to fit better
			},
		}),
		charts.WithYAxisOpts(opts.YAxis{
			Name: "Amount",
		}),
	)

	// Add data to scatter plot
	scatter.AddSeries("Amount", scatterData)//.
		// SetSeriesOptions(
		// 	charts.WithMarkPointNameTypeItemOpts(opts.MarkPointNameTypeItem{Name: "Max", Type: "max"}),
		// 	charts.WithMarkPointNameTypeItemOpts(opts.MarkPointNameTypeItem{Name: "Min", Type: "min"}),
		// )

	err := gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")
	if err != nil {
		fmt.Printf("Error: %+v\n", err)
	}
}

%%
plotSomething()